# Hướng dẫn sử dụng hệ thống RAG UIT@PubHealthQA

Notebook này hướng dẫn cách sử dụng hệ thống RAG đã được tái cấu trúc. Hệ thống RAG cho phép tìm kiếm và truy xuất thông tin từ các văn bản pháp luật y tế.

## 1. Cài đặt thư viện cần thiết

Đảm bảo bạn đã cài đặt các thư viện cần thiết:

In [12]:
# Cài đặt các thư viện cần thiết (chỉ chạy một lần)
# !pip install -U langchain faiss-cpu langchain-huggingface sentence-transformers tqdm pypdf

## 2. Import các thư viện và module

In [13]:
import json
import time
import logging
from pathlib import Path

# Import các module từ thư mục src
import sys
sys.path.append('..')

from src.utils.logging_utils import setup_logging
from src.embed.faiss_manager import initialize_embedding_model, load_vector_db
from src.retriever.faiss_retriever import query_documents

# Thiết lập logging
log_file_path = Path("../outputs/logs/rag_usage_log.txt")
setup_logging(log_file_path=log_file_path, console_output=True)

2025-04-21 22:43:38,196 - INFO - Đã thiết lập cấu hình logging.
2025-04-21 22:43:38,196 - INFO - File log: ..\outputs\logs\rag_usage_log.txt


## 3. Cấu hình các tham số

In [14]:
# Đường dẫn đến vector database
PERSIST_DIRECTORY = Path("../data/gold/db_faiss_phapluat_yte_full_final")
# Model embedding sử dụng
MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder"

print(f"Thư mục lưu Index FAISS: {PERSIST_DIRECTORY}")
print(f"Model Embedding: {MODEL_NAME}")

Thư mục lưu Index FAISS: ..\data\gold\db_faiss_phapluat_yte_full_final
Model Embedding: bkai-foundation-models/vietnamese-bi-encoder


## 4. Khởi tạo model embedding và tải vector database

In [15]:
# Khởi tạo model embedding
embeddings = initialize_embedding_model(model_name=MODEL_NAME)
if not embeddings:
    logging.error("Không thể khởi tạo model embedding. Hãy kiểm tra lại model_name hoặc cài đặt thư viện.")
    raise RuntimeError("Không thể khởi tạo model embedding.")

# Tải vector database
vectordb = load_vector_db(persist_directory=PERSIST_DIRECTORY, embeddings=embeddings)
if not vectordb:
    logging.error(f"Không thể tải vector database từ {PERSIST_DIRECTORY}. Kiểm tra lại đường dẫn hoặc chạy chunking.ipynb để tạo vector database.")
    raise RuntimeError("Không thể tải vector database.")

print(f"\nĐã tải vector database thành công. Số lượng vector: {vectordb.index.ntotal}")

2025-04-21 22:43:38,240 - INFO - Load pretrained SentenceTransformer: bkai-foundation-models/vietnamese-bi-encoder
2025-04-21 22:43:42,473 - INFO - Model Embedding 'bkai-foundation-models/vietnamese-bi-encoder' đã sẵn sàng.
2025-04-21 22:43:42,631 - INFO - Đã tải index FAISS thành công. Số lượng vector hiện tại: 15498



Đã tải vector database thành công. Số lượng vector: 15498


## 5. Thực hiện truy vấn cơ bản

In [16]:
def display_search_results(results, show_metadata=True):
    """Hiển thị kết quả tìm kiếm một cách rõ ràng."""
    if not results:
        print("Không tìm thấy kết quả nào.")
        return
    
    for i, result in enumerate(results):
        print(f"\n--- Kết quả {i+1} ---")
        
        # Kiểm tra xem result có phải là tuple (doc, score) không
        if isinstance(result, tuple) and len(result) == 2:
            doc, score = result
            print(f"Điểm số: {score:.4f}")
        else:
            doc = result
        
        # Hiển thị metadata nếu được yêu cầu
        if show_metadata and hasattr(doc, 'metadata') and doc.metadata:
            print("Metadata:")
            # Hiển thị các thông tin quan trọng
            doc_id = doc.metadata.get('document_id', 'N/A')
            doc_type = doc.metadata.get('document_type', 'N/A')
            effective_date = doc.metadata.get('effective_date', 'N/A')
            location = doc.metadata.get('location_detail', 'N/A')
            
            print(f"  - ID văn bản: {doc_id}")
            print(f"  - Loại văn bản: {doc_type}")
            print(f"  - Ngày hiệu lực: {effective_date}")
            print(f"  - Vị trí: {location}")
        
        # Hiển thị nội dung
        content = doc.page_content if hasattr(doc, 'page_content') else str(doc)
        print("\nNội dung:")
        print(content)
        print("-" * 80)

In [17]:
# Thử nghiệm truy vấn cơ bản
query = "Quy định về đăng ký thuốc mới"
k = 3  # Số lượng kết quả trả về

print(f"Câu hỏi: {query}")
print(f"Tìm kiếm {k} kết quả phù hợp nhất...")

start_time = time.time()
results = query_documents(vectordb, query, k=k)
end_time = time.time()

print(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")
display_search_results(results)

2025-04-21 22:43:42,686 - INFO - Đã tiền xử lý query: 'Quy định về đăng ký thuốc mới'
2025-04-21 22:43:42,775 - INFO - Truy vấn 'Quy định về đăng ký thuốc mới' hoàn tất sau 0.089 giây. Tìm thấy 3 kết quả.


Câu hỏi: Quy định về đăng ký thuốc mới
Tìm kiếm 3 kết quả phù hợp nhất...
Thời gian truy vấn: 0.095 giây

--- Kết quả 1 ---
Metadata:
  - ID văn bản: 08/2022/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2022-10-20
  - Vị trí: Chương I: QUY ĐỊNH CHUNG... -> Điều 9: Tiêu chí phân loại và các trường hợp công bố biệt dược gốc, sinh phẩm tham chiếu... -> Khoản 2 -> Điểm a

Nội dung:
- Thuốc được cấp giấy đăng ký lưu hành mới theo hình thức đăng ký lại quy định tại Thông tư số 44/2014/TT-BYT ngày 25 tháng 11 năm 2014 của Bộ trưởng Bộ Y tế quy định việc đăng ký thuốc (sau đây gọi tắt là Thông tư số 44/2014/TT-BYT) mà có cùng công thức bào chế, quy trình sản xuất, tiêu chuẩn chất lượng nguyên liệu, tiêu chuẩn chất lượng thuốc thành phẩm so với biệt dược gốc, sinh phẩm tham chiếu đã được công bố hoặc có thay đổi liên quan đến nội dung trên đã được Cục Quản lý Dược hoặc nước sở tại phê duyệt. Cơ sở đăng ký nộp hồ sơ cập nhật phân loại biệt dược gốc, sinh phẩm tham chiếu theo quy định tại

## 6. Truy vấn với điểm số

In [18]:
# Truy vấn với điểm số
query = "an toàn sinh học"
k = 5

print(f"Câu hỏi: {query}")
print(f"Tìm kiếm {k} kết quả phù hợp nhất với điểm số...")

start_time = time.time()
results = query_documents(vectordb, query, k=k, with_score=True)
end_time = time.time()

print(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")
display_search_results(results)

2025-04-21 22:43:42,786 - INFO - Đã tiền xử lý query: 'an toàn sinh học'
2025-04-21 22:43:42,838 - INFO - Truy vấn 'an toàn sinh học' hoàn tất sau 0.052 giây. Tìm thấy 5 kết quả.


Câu hỏi: an toàn sinh học
Tìm kiếm 5 kết quả phù hợp nhất với điểm số...
Thời gian truy vấn: 0.053 giây

--- Kết quả 1 ---
Điểm số: 34.0773
Metadata:
  - ID văn bản: 29/2012/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2013-02-01
  - Vị trí: Điều 7: Quy trình thẩm định cấp mới, cấp lại giấy chứng nhận an toàn sinh học... -> Khoản 3

Nội dung:
3. Quản lý giấy chứng nhận an toàn sinh học: a) Mỗi phòng xét nghiệm chỉ được cấp một giấy chứng nhận an toàn sinh học theo mẫu quy định tại Phụ lục 11 ban hành kèm theo Thông tư này; b) Hồ sơ đề nghị cấp mới, cấp lại giấy chứng nhận an toàn sinh học được lưu tại cơ quan cấp giấy chứng nhận; c) Sau khi cấp giấy chứng nhận an toàn sinh học cho cơ sở: - Bộ Y tế gửi văn bản thông báo cho Ủy ban nhân dân tỉnh và Sở Y tế tỉnh nơi cơ sở xét nghiệm đã được cấp giấy chứng nhận an toàn sinh học đặt trụ sở trong thời gian không quá 30 (ba mươ i) ngày kể từ ngày cấp mới, cấp lại giấy chứng nhận an toàn sinh học; - Sở Y tế tỉnh gửi văn bản thông báo c

## 7. Truy vấn với MMR (Maximum Marginal Relevance)

In [19]:
# Truy vấn với MMR (đa dạng hóa kết quả)
query = "Nội dung giấy chứng nhận cần có thông tin gì?"
k = 4
fetch_k = 20  # Số lượng kết quả để chọn lọc

print(f"Câu hỏi: {query}")
print(f"Tìm kiếm {k} kết quả đa dạng nhất (MMR) từ {fetch_k} kết quả ban đầu...")

start_time = time.time()
results = query_documents(vectordb, query, k=k, fetch_k=fetch_k, use_mmr=True)
end_time = time.time()

print(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")
display_search_results(results)

2025-04-21 22:43:42,850 - INFO - Đã tiền xử lý query: 'Nội dung giấy chứng nhận cần có thông tin gì?'


Câu hỏi: Nội dung giấy chứng nhận cần có thông tin gì?
Tìm kiếm 4 kết quả đa dạng nhất (MMR) từ 20 kết quả ban đầu...


2025-04-21 22:43:42,905 - INFO - Truy vấn 'Nội dung giấy chứng nhận cần có thông tin gì?' hoàn tất sau 0.055 giây. Tìm thấy 4 kết quả.


Thời gian truy vấn: 0.055 giây

--- Kết quả 1 ---
Metadata:
  - ID văn bản: 18/2019/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2019-07-17
  - Vị trí: Điều 4 -> Khoản 2

Nội dung:
2. Yêu cầu đối với nội dung của giấy chứng nhận, chứng nhận tương đương quy định tại các Điểm a, b và c Khoản 1 Điều này bao gồm các thông tin sau: a) Tên cơ quan, tổ chức có thẩm quyền cấp; b) Ngày cấp; c) Thời hạn hiệu lực (trong trường hợp giấy chứng nhận không ghi thời hạn hiệu lực thì phải có tài liệu chứng minh cơ sở sản xuất vẫn duy trì điều kiện an toàn thực phẩm: báo cáo đánh giá hoặc biên bản kiểm tra định kỳ); d) Họ tên, chữ ký của người cấp; đ) Tên, địa chỉ cơ sở được cấp; e) Phạm vi được chứng nhận; g) Dạng bào chế hoặc tên sản phẩm được chứng nhận.
--------------------------------------------------------------------------------

--- Kết quả 2 ---
Metadata:
  - ID văn bản: 22/2011/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2011-07-25
  - Vị trí: Chương III: HOẠT ĐỘNG CHÍNH CỦA 

## 8. Giao diện truy vấn tương tác

In [20]:
def interactive_search():
    """Giao diện tìm kiếm tương tác."""
    print("\n=== GIAO DIỆN TÌM KIẾM VĂN BẢN PHÁP LUẬT Y TẾ ===\n")
    print("Nhập 'exit' để thoát.")
    
    while True:
        query = input("\nNhập câu hỏi của bạn: ")
        if query.lower() == 'exit':
            print("Tạm biệt!")
            break
            
        # Lấy tùy chọn
        try:
            k = int(input(f"Số lượng kết quả (mặc định: 3): ") or "3")
            use_mmr = input("Sử dụng MMR để đa dạng hóa kết quả? (y/n, mặc định: n): ").lower() == 'y'
            
            if use_mmr:
                fetch_k = int(input(f"fetch_k cho MMR (mặc định: {k*5}): ") or str(k*5))
            else:
                fetch_k = k*5
                
            show_score = input("Hiển thị điểm số? (y/n, mặc định: n): ").lower() == 'y'
            
            # Thực hiện truy vấn
            print(f"\nĐang tìm kiếm cho câu hỏi: '{query}'...")
            start_time = time.time()
            
            results = query_documents(
                vectordb, 
                query, 
                k=k, 
                fetch_k=fetch_k, 
                use_mmr=use_mmr, 
                with_score=show_score
            )
            
            end_time = time.time()
            print(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")
            
            # Hiển thị kết quả
            display_search_results(results)
            
        except Exception as e:
            print(f"Lỗi: {e}")
            logging.error(f"Lỗi khi thực hiện truy vấn: {e}", exc_info=True)

In [21]:
# Chạy giao diện tìm kiếm tương tác
interactive_search()


=== GIAO DIỆN TÌM KIẾM VĂN BẢN PHÁP LUẬT Y TẾ ===

Nhập 'exit' để thoát.


2025-04-21 22:43:52,966 - INFO - Đã tiền xử lý query: 'Trình tự thực hiện tự công bố cơ sở đủ điều kiện tiêm chủng như thế nào?'
2025-04-21 22:43:53,034 - INFO - Truy vấn 'Trình tự thực hiện tự công bố cơ sở đủ điều kiện tiêm chủng như thế nào?' hoàn tất sau 0.068 giây. Tìm thấy 10 kết quả.



Đang tìm kiếm cho câu hỏi: 'Trình tự thực hiện tự công bố cơ sở đủ điều kiện tiêm chủng như thế nào?'...
Thời gian truy vấn: 0.069 giây

--- Kết quả 1 ---
Metadata:
  - ID văn bản: 30/2024/TT-BYT
  - Loại văn bản: Thông tư
  - Ngày hiệu lực: 2024-12-19
  - Vị trí: Điều 5: Điều khoản tham chiếu... -> Khoản 5

Nội dung:
Dịch vụ 25 Tổ chức tiêm chủng: tư vấn trước tiêm chủng, khám sàng lọc, tiêm chủng, theo dõi phản ứng sau tiêm chủng, quản lý đối tượng, thống kê báo cáo 1. Địa điểm thực hiện: tại Trạm Y tế xã hoặc tại cơ sở y tế hoặc điểm cung cấp dịch vụ tiêm chủng tại cộng đồng hoặc tại nhà. 2. Đối tượng: trẻ em và phụ nữ có thai là đối tượng của Chương trình Tiêm chủng mở rộng trên toàn quốc. 3. Tần suất thực hiện: hằng tháng. 4. Nội dung: - Trước tiêm chủng: điều tra, lập danh sách đối tượng thuộc Chương trình tiêm chủng mở rộng; thông báo cho đối tượng tiêm chủng để tham gia tiêm chủng đúng lịch, đủ liều; khám sàng lọc; tư vấn trước tiêm chủng; - Thực hiện tiêm chủng: thực hiện kiể

## 9. Kết hợp với LLM để xây dựng hệ thống RAG hoàn chỉnh

Để tạo một hệ thống RAG hoàn chỉnh, bạn có thể kết hợp kết quả truy xuất với một Large Language Model (LLM). Dưới đây là ví dụ cách tích hợp với LLM từ OpenAI hoặc mô hình mã nguồn mở.

In [22]:
# Ví dụ tích hợp với LLM (yêu cầu cài đặt thêm thư viện)
# !pip install langchain-openai  # Cho OpenAI
# !pip install langchain-community  # Cho các mô hình mã nguồn mở

# Nhận xét các dòng sau nếu chưa cài đặt thư viện cần thiết
'''
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# Đặt API key (thay thế bằng API key của bạn hoặc sử dụng biến môi trường)
import os
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Khởi tạo LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Tạo retriever từ vector database
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# Tạo chuỗi RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True
)

# Sử dụng
query = "Quy định về đăng ký thuốc mới tại Việt Nam?"
result = qa_chain({"query": query})
print("Câu trả lời:")
print(result["result"])
'''

'\nfrom langchain_openai import ChatOpenAI\nfrom langchain.chains import RetrievalQA\n\n# Đặt API key (thay thế bằng API key của bạn hoặc sử dụng biến môi trường)\nimport os\nos.environ["OPENAI_API_KEY"] = "your-api-key-here"\n\n# Khởi tạo LLM\nllm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)\n\n# Tạo retriever từ vector database\nretriever = vectordb.as_retriever(search_kwargs={"k": 5})\n\n# Tạo chuỗi RetrievalQA\nqa_chain = RetrievalQA.from_chain_type(\n    llm=llm,\n    chain_type="stuff",\n    retriever=retriever,\n    return_source_documents=True,\n    verbose=True\n)\n\n# Sử dụng\nquery = "Quy định về đăng ký thuốc mới tại Việt Nam?"\nresult = qa_chain({"query": query})\nprint("Câu trả lời:")\nprint(result["result"])\n'